In [22]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *

import sklearn 
from sklearn.linear_model import LogisticRegression

## 1
Существуют две версии одновыборочного Z-критерия для доли, отличающихся формулами для статистики. Одна из этих версий более точная, 
поэтому мы говорили только о ней, а вторую не обсуждали вообще. Какую из этих двух версий одновыборочных Z-критериев для доли мы проходили?

**Ответ**: Версия множителей Лагранжа со статистикой $Z=\frac{\hat{p}-p_0}{\sqrt{\frac{p_0\left(1-p_0\right)}{n}}}$

## 2
Z-критерий для двух долей в связанных выборках использует только информацию о:

**Ответ**: количестве совпадающих пар значений признаков — $(0, 0)$ и $(1, 1)$ и объеме выборок $n$

## 3

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [2]:
n = 50
n1 = 34
n2 = 16

s1 = 10
s2 = 4

In [3]:
def proportions_diff_z_stat_ind(s1, s2, n1, n2): 
    p1 = s1 / n1
    p2 = s2 / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [4]:
z = proportions_diff_z_stat_ind(s1, s2, n1, n2); z

0.32410186177608225

In [5]:
round(1 - stats.norm.cdf(z), 4)

0.3729

**Ответ**: 0.3729

## 4

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

Загрузите данные: banknotes.txt

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции `sklearn.cross_validation.train_test_split` (зафиксируйте `random state = 1`). На оставшихся 150 настройте два классификатора поддельности банкнот:

* логистическая регрессия по признакам $X_1,X_2,X_3$;
* логистическая регрессия по признакам $X_4,X_5,X_6$.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5 \times 10^{-8}$ , нужно ввести 8).
 

In [6]:
banknotes = pd.read_csv('banknotes.txt', sep='\t'); banknotes.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [28]:
X = banknotes.drop(['real'], axis=1)
y = banknotes.real

In [29]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=.5, random_state=1)

X_train_1 = X_train[['X1', 'X2', 'X3']]
X_train_2 = X_train[['X4', 'X5', 'X6']]

X_test_1 = X_test[['X1', 'X2', 'X3']]
X_test_2 = X_test[['X4', 'X5', 'X6']]

In [30]:
len(y_test) / len(y)

0.5

`sklearn.linear_model.LogisticRegression(..., solver='lbfgs')` — Logistic Regression (aka logit, MaxEnt) classifier.

`solver` – Algorithm to use in the optimization problem

In [42]:
lr_1 = LogisticRegression(solver='lbfgs')
lr_1.fit(X_train_1, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
lr_1.score(X_test_1, y_test)

0.84

In [44]:
sum(abs(y_test - lr_1.predict(X_test_1)))

16

In [45]:
lr_2 = LogisticRegression(solver='lbfgs')
lr_2.fit(X_train_2, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
lr_2.score(X_test_2, y_test)

0.99

In [47]:
sum(abs(y_test  - lr_2.predict(X_test_2)))

1

In [54]:
def proportions_diff_z_stat_ind(s1, s2, n1, n2): 
    p1 = s1 / n1
    p2 = s2 / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [59]:
z = proportions_diff_z_stat_ind(sum(abs(y_test - lr_1.predict(X_test_1))), sum(abs(y_test  - lr_2.predict(X_test_2))), X_test_1.shape[0], X_test_2.shape[0]); z

3.8032620622786917

In [60]:
1 - stats.norm.cdf(z)

7.140157481244636e-05

**Ответ**: 5

## 5
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [ ]:
# ?